# BYOC (Bring Your Own Compute) Environments

Learn how to run Claude agent workloads on your own infrastructure using BYOC (Bring Your Own Compute) environments. This cookbook covers creating self-hosted environments and running the official Environment Runner.

> **Note:** The BYOC API is currently in beta. API signatures and features may change.

## Table of Contents

1. [Overview & Architecture](#overview)
2. [Setup & Installation](#setup)
3. [Creating BYOC Environments](#environments)
4. [Running the Environment Runner](#worker)
5. [Best Practices](#best-practices)

## 1. Overview & Architecture {#overview}

### What is BYOC?

BYOC (Bring Your Own Compute) allows you to run agent workloads on your own infrastructure while using Anthropic's API for orchestration. This provides:

- **Full control** over compute resources and security
- **Data residency** - keep data within your infrastructure
- **Custom scaling** - scale based on your workload patterns
- **Integration** with existing infrastructure

### Architecture

In a BYOC setup, you run the **Environment Runner** on your infrastructure:

```
 Anthropic Cloud          Your Infrastructure
┌──────────────────┐     ┌──────────────────┐
│                  │     │                  │
│ ┌──────────────┐ │     │ ┌──────────────┐ │
│ │ Work Queue   │◄├─────┤►│ Environment  │ │
│ └──────────────┘ │Polls│ │   Runner     │ │
│                  │     │ └──────┬───────┘ │
│ ┌──────────────┐ │     │        │         │
│ │Session State │ │     │        ▼         │
│ │   (API)      │ │     │ ┌──────────────┐ │
│ └──────────────┘ │     │ │ Claude Code  │ │
│                  │     │ └──────────────┘ │
└──────────────────┘     └──────────────────┘
```

The Environment Runner:
1. **Polls** the work queue for new session tasks
2. **Launches** Claude Code to process each session
3. **Manages** heartbeats and work item lifecycle
4. **Communicates** results back via Anthropic's session ingress API

## 2. Setup & Installation {#setup}

### Prerequisites

- Python 3.8 or higher
- Anthropic API key with BYOC access from [console.anthropic.com](https://console.anthropic.com/)
- [Claude Code](https://docs.anthropic.com/en/docs/claude-code) CLI installed on your runner infrastructure
- Infrastructure to run the BYOC worker (VM, container, etc.)

In [ ]:
# Install dependencies
%pip install anthropic python-dotenv --quiet

In [1]:
import os
import time

import anthropic

# Verify SDK version (requires 0.75.0 or higher)
print(f"Anthropic SDK version: {anthropic.__version__}")

Anthropic SDK version: 0.75.0


### Configure API Key

In [2]:
from dotenv import load_dotenv

load_dotenv()

# Initialize the client with beta headers
client = anthropic.Anthropic(
    default_headers={
        "anthropic-beta": "environments-2025-12-12,agent-api-2025-12-12",
    },
)

# Verify connection
if os.getenv("ANTHROPIC_API_KEY"):
    print("API key configured successfully")
    print(f"Base URL: {client.base_url}")
else:
    print("Warning: ANTHROPIC_API_KEY not found in environment")

API key configured successfully
Base URL: https://api.anthropic.com


## 3. Creating BYOC Environments {#environments}

BYOC environments use `type: "self_hosted"` configuration. Unlike cloud environments, you manage the compute infrastructure yourself.

### Environment Service Key

The Environment Runner requires an **Environment Service Key** for authentication:

1. Create a BYOC environment (via API or Console)
2. In the [Console](https://console.anthropic.com), navigate to your environment
3. Click **"Generate secret key"** to create a service key
4. Set the key as `ANTHROPIC_SERVICE_KEY` environment variable (used later by the runner)

In [5]:
import uuid

# Name prefix for BYOC environments created by this cookbook
# This helps identify and reuse environments in shared workspaces
BYOC_ENV_NAME_PREFIX = "byoc-cookbook-"


def get_or_create_byoc_environment():
    """
    Get an existing cookbook BYOC environment or create a new one.
    Only looks for environments with the cookbook's name prefix to avoid
    accidentally using environments created by others in shared workspaces.
    """
    # Look for an existing BYOC environment created by this cookbook
    existing = client.beta.environments.list()
    for env in existing.data:
        if env.name.startswith(BYOC_ENV_NAME_PREFIX):
            # Verify it's actually a self_hosted environment
            if hasattr(env.config, "type") and env.config.type == "self_hosted":
                print(f"Using existing cookbook BYOC environment: {env.id}")
                print(f"  Name: {env.name}")
                return env

    # Create a new BYOC environment
    print("Creating new BYOC environment...")
    environment = client.beta.environments.create(
        name=f"{BYOC_ENV_NAME_PREFIX}{uuid.uuid4().hex[:8]}",
        description="BYOC environment for cookbook demonstration",
        config={
            "type": "self_hosted",
        },
        scope="organization",  # Visible to all accounts in org
        metadata={
            "datacenter": "us-east-1",
            "purpose": "cookbook-demo",
        },
    )

    print(f"Created BYOC environment: {environment.id}")
    print(f"  Name: {environment.name}")
    print(f"  Scope: {environment.scope}")
    return environment


byoc_env = get_or_create_byoc_environment()

Using existing cookbook BYOC environment: env_01Dm2rmdx8HnuJQVWyj2Zfdv
  Name: byoc-cookbook-715800bf


### Listing Cookbook BYOC Environments

View BYOC environments created by this cookbook:

In [6]:
# List BYOC environments created by this cookbook
all_environments = client.beta.environments.list()
cookbook_byoc_environments = [
    env for env in all_environments.data 
    if env.name.startswith(BYOC_ENV_NAME_PREFIX) 
    and hasattr(env.config, "type") 
    and env.config.type == "self_hosted"
]

print(f"Cookbook BYOC Environments (prefix: '{BYOC_ENV_NAME_PREFIX}'):")
print("=" * 70)

if not cookbook_byoc_environments:
    print("\nNo cookbook BYOC environments found. Run the cell above to create one.")
else:
    for env in cookbook_byoc_environments:
        print(f"\nID: {env.id}")
        print(f"  Name: {env.name}")
        print(f"  State: {env.state}")
        print(f"  Type: {env.config.type}")

    print("\n" + "=" * 70)
    print(f"Total: {len(cookbook_byoc_environments)} cookbook BYOC environment(s)")

Cookbook BYOC Environments (prefix: 'byoc-cookbook-'):

ID: env_01Dm2rmdx8HnuJQVWyj2Zfdv
  Name: byoc-cookbook-715800bf
  State: active
  Type: self_hosted

Total: 1 cookbook BYOC environment(s)


### Environment Scopes

| Scope | Description |
|-------|-------------|
| `organization` | Visible to all accounts in the organization |
| `account` | Restricted to the owning account only |

## 4. Running the Environment Runner {#worker}

Anthropic provides an official **Environment Runner** that handles the complete BYOC workflow:
- Polls the work queue for sessions
- Executes sessions on your infrastructure
- Manages heartbeats and work item lifecycle
- Calls Anthropic's inference API for model responses

### Installing the Environment Runner

Download the runner to the current directory (no system-wide installation required):

In [5]:
import subprocess
import glob as glob_module

# Check if runner is already installed
runner_dirs = glob_module.glob("claude-environment-runner_*")
if runner_dirs:
    # Find the binary in existing installation
    runner_binary = glob_module.glob("claude-environment-runner_*/claude-environment-runner")
    if runner_binary:
        RUNNER_PATH = runner_binary[0]
        print(f"Environment runner already installed: {RUNNER_PATH}")
    else:
        print("Runner directory exists but binary not found. Re-downloading...")
        !curl -fsSL https://console.anthropic.com/download-environment-runner.sh | bash
        runner_binary = glob_module.glob("claude-environment-runner_*/claude-environment-runner")
        RUNNER_PATH = runner_binary[0] if runner_binary else None
else:
    # Download the runner
    print("Downloading environment runner...")
    !curl -fsSL https://console.anthropic.com/download-environment-runner.sh | bash
    runner_binary = glob_module.glob("claude-environment-runner_*/claude-environment-runner")
    RUNNER_PATH = runner_binary[0] if runner_binary else None

if RUNNER_PATH:
    # Verify it works
    result = subprocess.run([RUNNER_PATH, "--version"], capture_output=True, text=True)
    print(f"Runner version: {result.stdout.strip()}")

Environment runner already installed: claude-environment-runner_0.0.2_darwin_arm64/claude-environment-runner
Runner version: environment-runner version 0.0.2


### Preparing the Service Key

The runner needs the service key saved to a file:

In [6]:
# Load service key from environment variable
SERVICE_KEY = os.getenv("ANTHROPIC_SERVICE_KEY")

# Save service key to a file for the runner
SERVICE_KEY_FILE = "environment_service_key.txt"

if SERVICE_KEY:
    with open(SERVICE_KEY_FILE, "w") as f:
        f.write(SERVICE_KEY)
    print(f"Service key saved to: {SERVICE_KEY_FILE}")
else:
    print("ERROR: No service key available!")
    print("Set ANTHROPIC_SERVICE_KEY environment variable first.")

ERROR: No service key available!
Set ANTHROPIC_SERVICE_KEY environment variable first.


### Starting the Runner

The runner uses `orchestrator` mode to poll for work and process sessions. Key flags:

| Flag | Description |
|------|-------------|
| `--organization-id` | Your Anthropic organization ID |
| `--environment-id` | The BYOC environment to serve |
| `--service-key-file` | Path to the environment service key file |
| `--sandbox-backend` | Sandbox type (`none` for macOS dev, `srt` for Linux production) |

> **Note:** On macOS, use `--sandbox-backend=none` since the Linux sandbox runtime (`srt`) is not available. In production Linux environments, omit this flag to use the default sandbox for security isolation.

In [ ]:
# Get organization ID from the environment (needed for runner)
ORG_ID = os.getenv("ANTHROPIC_ORG_ID")

if not ORG_ID:
    print("Note: ANTHROPIC_ORG_ID not set.")
    print("You can find your organization ID in the Console URL or environment details.")
    print("Set it with: os.environ['ANTHROPIC_ORG_ID'] = 'your-org-id'")

# Build the runner command
runner_cmd = [
    RUNNER_PATH,
    "orchestrator",
    f"--organization-id={ORG_ID}" if ORG_ID else "",
    f"--environment-id={byoc_env.id}",
    f"--service-key-file={SERVICE_KEY_FILE}",
    "--sandbox-backend=none",  # Required on macOS; omit on Linux for sandbox isolation
]
# Remove empty strings
runner_cmd = [c for c in runner_cmd if c]

print("Runner command:")
print(" ".join(runner_cmd))

In [ ]:
# Start the runner in the background
runner_process = None

if ORG_ID and RUNNER_PATH and SERVICE_KEY:
    print("Starting environment runner...")
    runner_process = subprocess.Popen(
        runner_cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
    )
    print(f"Runner started with PID: {runner_process.pid}")
    
    # Give it a moment to start
    time.sleep(2)
    
    # Check if it's still running
    if runner_process.poll() is None:
        print("Runner is running and polling for work...")
    else:
        print(f"Runner exited with code: {runner_process.returncode}")
        output, _ = runner_process.communicate()
        print(f"Output: {output[:1000] if output else 'None'}")
else:
    print("Cannot start runner - missing required configuration:")
    if not ORG_ID:
        print("  - ANTHROPIC_ORG_ID not set")
    if not RUNNER_PATH:
        print("  - Runner not installed")
    if not SERVICE_KEY:
        print("  - Service key not configured")

### Demo: Create and Interact with a Session

With the runner active, create a session and send a message. The runner picks up the session, launches Claude Code to process it, and streams the response back:

In [7]:
# Create a session in the BYOC environment and interact with it
# The runner will pick this up, launch Claude Code, and process the session

print("Creating session in BYOC environment...")
print("=" * 60)

session = client.beta.sessions.create(
    environment=byoc_env.id,
    agent={
        "model": "claude-sonnet-4-5-20250929",
        "system": "You are a helpful assistant. Keep responses brief (2-3 sentences).",
    },
    title="BYOC Demo Session",
)

print(f"Session created: {session.id}")
print(f"Initial status: {session.status}")

# Poll session status until the runner picks it up
print("\nWaiting for runner to pick up the session...")
max_wait = 120  # seconds
start_time = time.time()

while time.time() - start_time < max_wait:
    session_status = client.beta.sessions.retrieve(session_id=session.id)
    if session_status.status in ("idle", "running"):
        print(f"Session is {session_status.status}!")
        break
    time.sleep(2)
else:
    print(f"Timeout waiting for session (status: {session_status.status})")

# Send a message and stream the response
if session_status.status in ("idle", "running"):
    print("\nSending user message and streaming response...")
    print("-" * 60)

    with client.beta.sessions.stream(session_id=session.id) as stream:
        # Send a user message to the session
        client.beta.sessions.events.send(
            session_id=session.id,
            events=[{
                "type": "user",
                "content": [{"type": "text", "text": "Hello! What's 2 + 2? Please answer briefly."}]
            }]
        )

        # Stream events from the session
        for event in stream:
            if event.type == "agent":
                # Print the agent's response text
                for block in event.content:
                    if hasattr(block, "text"):
                        print(block.text, end="", flush=True)
            elif event.type == "status_idle":
                # Agent finished responding
                break

    print("\n" + "-" * 60)
    print("Session completed successfully!")
    print("=" * 60)

Creating session in BYOC environment...
Session created: session_01PU3KuQDyXEqHKMCBGhENz2
Initial status: running

Waiting for runner to pick up the session...
Session is running!

Sending user message and streaming response...
------------------------------------------------------------
2 + 2 = 4
------------------------------------------------------------
Session completed successfully!


### Stopping the Runner

Stop the environment runner when you're done:

In [ ]:
# Stop the runner
if runner_process and runner_process.poll() is None:
    print("Stopping environment runner...")
    runner_process.terminate()
    
    # Wait for graceful shutdown
    try:
        runner_process.wait(timeout=5)
        print("Runner stopped gracefully")
    except subprocess.TimeoutExpired:
        print("Runner didn't stop gracefully, forcing...")
        runner_process.kill()
        runner_process.wait()
        print("Runner killed")
else:
    print("Runner is not running")

## 5. Best Practices {#best-practices}

### Scaling

1. **Monitor queue depth** - Scale runners based on pending work
2. **Use multiple runners** - Run multiple runner instances for parallelism
3. **Set appropriate timeouts** - Balance responsiveness vs. API load

### Security

1. **Isolate execution** - Run sessions in isolated containers
2. **Limit network access** - Configure firewalls for session containers
3. **Rotate credentials** - Regularly rotate service keys

### Production Checklist

| Item | Description |
|------|-------------|
| Logging | Enable runner logging with `--log-level=debug` |
| Metrics | Monitor queue depth, processing time, error rates |
| Alerts | Alert on queue backup, runner failures |
| Health checks | Monitor runner process health |
| Graceful shutdown | Handle SIGTERM, let runner finish in-flight work |

## Cleanup

Clean up resources created in this cookbook:

In [ ]:
import shutil

# Uncomment to clean up:

# 1. Archive sessions created in this cookbook
# sessions = client.beta.sessions.list()
# for s in sessions.data:
#     if hasattr(s, "environment") and s.environment == byoc_env.id:
#         client.beta.sessions.archive(session_id=s.id)
#         print(f"Archived session: {s.id}")

# 2. Remove the environment runner (downloaded to current directory)
# for runner_dir in glob_module.glob("claude-environment-runner_*"):
#     shutil.rmtree(runner_dir)
#     print(f"Removed: {runner_dir}")

# 3. Remove the service key file
# if os.path.exists(SERVICE_KEY_FILE):
#     os.remove(SERVICE_KEY_FILE)
#     print(f"Removed: {SERVICE_KEY_FILE}")

# 4. Archive the BYOC environment (or delete permanently)
# client.beta.environments.archive(environment_id=byoc_env.id)
# print(f"Archived environment: {byoc_env.id}")
#
# client.beta.environments.delete(environment_id=byoc_env.id, force=True)
# print(f"Deleted environment: {byoc_env.id}")

## Next Steps

Now that you understand BYOC:

1. **Deploy to production** - Run the Environment Runner on your infrastructure
2. **Add monitoring** - Integrate with your observability stack
3. **Implement auto-scaling** - Scale runners based on queue depth

### Related Cookbooks

- [Agent API](./agent_api.ipynb) - Getting started with Anthropic Cloud environments

### Resources

- [Anthropic API Documentation](https://docs.anthropic.com/)
- [Anthropic Console](https://console.anthropic.com/)